In [4]:
#!pip install jupyter-dash

In [5]:
#!pip install dash-bootstrap-components

Importing the necessary libraries

In [2]:
from dash import Dash, dcc, html, Input, Output, State, ctx
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.io as pio
from datetime import datetime
import smtplib
from email.message import EmailMessage
import os

# ---------- Load Data ----------
DATA_FOLDER = os.path.expanduser("~/Desktop/Python/Machine Learning")
DATA_FILE = os.path.join(DATA_FOLDER, "employee_performance.csv")

def load_data():
    try:
        df = pd.read_csv(DATA_FILE, low_memory=False)
        df = df.drop(columns=[col for col in df.columns if "Unnamed" in col or df[col].isnull().all()])
        df['Hire_Date'] = pd.to_datetime(df['Hire_Date'], dayfirst=True)
        return df.dropna(subset=['Hire_Date'])
    except Exception as e:
        print("Data loading error:", e)
        return pd.DataFrame()

# ---------- Email ----------
email_status = "📭 No email sent yet"
dashboard_status = "📊 Dashboard not updated yet"

def send_email():
    global email_status
    try:
        df = load_data()
        threshold = 1.5
        if 'Retension risk index' not in df.columns:
            email_status = "❌ Error: 'Retension risk index' column not found in data."
            return

        risky_employees = df[df['Retension risk index'] > threshold]
        if risky_employees.empty:
            email_status = f"✅ No risky employees found with risk < {threshold}"
            return

        selected_cols = ['Employee_ID', 'Department', 'Retension risk index']
        risky_employees = risky_employees[selected_cols]
        table_str = risky_employees.to_string(index=False)

        msg = EmailMessage()
        msg['Subject'] = '⚠️ Retention Risk Alert - At-Risk Employees'
        msg['From'] = 'kanchanaskrp@gmail.com'
        msg['To'] = 'kanchanaskrp@gmail.com'
        msg.set_content(f'''
Hello,

This is an automated alert for employees at risk of leaving the company.
Below are employees with a Retention Risk Index below {threshold}:

{table_str}

Regards,
Employee Dashboard Bot
''')

        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login('kanchanaskrp@gmail.com', 'ugscscvebllvhvii')
            smtp.send_message(msg)

        email_status = f"📧 Email sent at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    except Exception as e:
        email_status = f"❌ Email failed: {e}"

# ---------- App ----------
pio.templates.default = "plotly_dark"
df = load_data()

departments = [{'label': dept, 'value': dept} for dept in sorted(df['Department'].dropna().unique())]
job_titles = [{'label': job, 'value': job} for job in sorted(df['Job_Title'].dropna().unique())]
years = sorted(df['Hire_Date'].dt.year.unique())
months = list(range(1, 13))
days = list(range(1, 32))

app = Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

# ---------- Predefined Q&A ----------
predefined_qa = {
    "What is the retention risk threshold?": "The threshold is 1.5. Employees above this index are considered at risk.",
    "How is performance score calculated?": "It’s an internal metric based on targets, KPIs, and manager feedback.",
    "What is the average satisfaction score?": "It’s calculated from the 'Employee_Satisfaction_Score' column.",
    "What does remote work frequency mean?": "It indicates how often each department works remotely.",
    "Who gets the email alerts?": "Emails are sent to kanchanaskrp@gmail.com."
}

# ---------- Layout ----------
app.layout = dbc.Container([
    html.H4("🤖 Employee Dashboard Chatbot", className="text-center mt-4 mb-3"),
    dbc.Row([
        dbc.Col([
            html.Label("Choose a predefined question:", style={"color": "white"}),
            dcc.Dropdown(
                id='faq-dropdown',  # ✅ ID added here
                placeholder="Select a question...",
                options=[{'label': q, 'value': q} for q in predefined_qa.keys()],
                className="mb-3"
            ),

            html.Label("Or ask your own question:", style={"color": "white"}),
            dbc.InputGroup([
                dbc.Input(id='chat-input', placeholder="Type or speak your question...", type='text'),
                dbc.Button("🎤 Speak", id='voice-btn', color='info', n_clicks=0)
            ], className="mb-2"),
            dbc.Row([
                dbc.Col(dbc.Button("Submit", id='chat-submit-btn', color='primary', className='me-2'), width="auto"),
                dbc.Col(dbc.Button("Clear", id='chat-clear-btn', color='secondary'), width="auto")
            ], className="mb-2"),
            dbc.Alert(id='chat-response', children="Welcome! Ask me anything about the dashboard.", color="secondary")
        ], md=12)
    ]),

    html.H2("Employee Performance Dashboard", className="text-center my-4"),

    dbc.Row([
        dbc.Col(dbc.Alert(id='dashboard-alert', color="info", className="text-center"), md=6),
        dbc.Col([
            dbc.Button("Send Retention Email", id='send-email-btn', color='danger', className="mb-2"),
            dbc.Alert(id='email-alert', color="success", className="mt-2")
        ], md=6)
    ], className="mb-4"),

    dcc.Interval(id='interval-component', interval=60 * 1000, n_intervals=0),

    dbc.Row([
        dbc.Col([
            html.Label("Select Department", style={"color": "white"}),
            dcc.Dropdown(id='dept-filter', options=[{'label': 'All', 'value': 'All'}] + departments, value='All')
        ], md=3),
        dbc.Col([
            html.Label("Select Job Title", style={"color": "white"}),
            dcc.Dropdown(id='job-filter', options=[{'label': 'All', 'value': 'All'}] + job_titles, value='All')
        ], md=3),
        dbc.Col([
            html.Label("Start Date", style={"color": "white"}),
            dcc.Dropdown(id='start-year', options=[{'label': y, 'value': y} for y in years], placeholder="Year"),
            dcc.Dropdown(id='start-month', options=[{'label': m, 'value': m} for m in months], placeholder="Month"),
            dcc.Dropdown(id='start-day', options=[{'label': d, 'value': d} for d in days], placeholder="Day")
        ], md=3),
        dbc.Col([
            html.Label("End Date", style={"color": "white"}),
            dcc.Dropdown(id='end-year', options=[{'label': y, 'value': y} for y in years], placeholder="Year"),
            dcc.Dropdown(id='end-month', options=[{'label': m, 'value': m} for m in months], placeholder="Month"),
            dcc.Dropdown(id='end-day', options=[{'label': d, 'value': d} for d in days], placeholder="Day")
        ], md=3)
    ], className='mb-4'),

    dbc.Row([
        dbc.Col(dbc.Card([dbc.CardBody([html.H6("Total Employees", className="text-center"), html.H3(id='kpi-employees', className="text-center")])]), md=4),
        dbc.Col(dbc.Card([dbc.CardBody([html.H6("Avg. Productivity", className="text-center"), html.H3(id='kpi-performance', className="text-center")])]), md=4),
        dbc.Col(dbc.Card([dbc.CardBody([html.H6("Avg. Satisfaction", className="text-center"), html.H3(id='kpi-satisfaction', className="text-center")])]), md=4)
    ]),
    dbc.Row([
        dbc.Col(dbc.Card([dbc.CardBody([html.H6("Avg. Age", className="text-center"), html.H3(id='kpi-age', className="text-center")])]), md=4),
        dbc.Col(dbc.Card([dbc.CardBody([html.H6("Avg. Salary", className="text-center"), html.H3(id='kpi-annual-salary', className="text-center")])]), md=4),
        dbc.Col(dbc.Card([dbc.CardBody([html.H6("Avg. Tenure", className="text-center"), html.H3(id='kpi-years', className="text-center")])]), md=4)
    ], className='mb-4'),

    dbc.Row([
        dbc.Col(dcc.Graph(id='donut-remote-work'), md=6),
        dbc.Col(dcc.Graph(id='bar-projects-jobtitle'), md=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='line-years-performance'), md=6),
        dbc.Col(dcc.Graph(id='scatter-age-performance'), md=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='box-dept-satisfaction'), md=6),
        dbc.Col(dcc.Graph(id='heatmap-performance-satisfaction'), md=6)
    ])
], fluid=True)
# Add voice recognition script
app.clientside_callback(
    """
    function(n_clicks) {
        if (!('webkitSpeechRecognition' in window)) {
            alert("Voice recognition not supported in this browser.");
            return window.dash_clientside.no_update;
        }

        const recognition = new webkitSpeechRecognition();
        recognition.lang = 'en-US';
        recognition.interimResults = false;
        recognition.maxAlternatives = 1;

        recognition.onresult = function(event) {
            const transcript = event.results[0][0].transcript;
            const input = document.getElementById('chat-input');
            input.value = transcript;
            input.dispatchEvent(new Event('input', { bubbles: true }));
        };

        recognition.onerror = function(event) {
            alert("Error during voice recognition: " + event.error);
        };

        recognition.start();
        return "";
    }
    """,
    Output('voice-btn', 'children'),  # Dummy output, not used
    Input('voice-btn', 'n_clicks')
)

# ---------- Dashboard Update ----------
@app.callback(
    Output('kpi-employees', 'children'),
    Output('kpi-performance', 'children'),
    Output('kpi-satisfaction', 'children'),
    Output('kpi-age', 'children'),
    Output('kpi-annual-salary', 'children'),
    Output('kpi-years', 'children'),
    Output('donut-remote-work', 'figure'),
    Output('bar-projects-jobtitle', 'figure'),
    Output('line-years-performance', 'figure'),
    Output('scatter-age-performance', 'figure'),
    Output('box-dept-satisfaction', 'figure'),
    Output('heatmap-performance-satisfaction', 'figure'),
    Input('dept-filter', 'value'),
    Input('job-filter', 'value'),
    Input('start-year', 'value'),
    Input('start-month', 'value'),
    Input('start-day', 'value'),
    Input('end-year', 'value'),
    Input('end-month', 'value'),
    Input('end-day', 'value'),
)
def update_dashboard(dept, job, sy, sm, sd, ey, em, ed):
    dff = load_data()
    if dept != 'All': dff = dff[dff['Department'] == dept]
    if job != 'All': dff = dff[dff['Job_Title'] == job]
    try:
        if all(v is not None for v in [sy, sm, sd, ey, em, ed]):
            start_date = datetime(sy, sm, sd)
            end_date = datetime(ey, em, ed)
            dff = dff[(dff['Hire_Date'] >= start_date) & (dff['Hire_Date'] <= end_date)]
    except: pass

    kpi1 = len(dff)
    kpi2 = round(dff['Productivity score'].mean(), 2)
    kpi3 = round(dff['Employee_Satisfaction_Score'].mean(), 2)
    kpi4 = round(dff['Age'].mean(), 1)
    kpi5 = f"${round(dff['Annual Salary'].mean(), 2):,}"
    kpi6 = round(dff['Years_At_Company'].mean(), 2)

    fig_donut = px.pie(dff, names='Department', values='Remote_Work_Frequency', hole=0.5, title='Remote Work Frequency by Department', color_discrete_sequence=px.colors.sequential.RdBu)
    fig_bar = px.bar(dff.groupby('Job_Title')['Projects_Handled'].sum().reset_index(), x='Job_Title', y='Projects_Handled', title='Projects Handled by Job Title', color='Projects_Handled', color_continuous_scale='Bluered')
    fig_line = px.line(dff.groupby('Years_At_Company')['Performance_Score'].mean().reset_index(), x='Years_At_Company', y='Performance_Score', markers=True, title='Performance Over Years at Company', color_discrete_sequence=['cyan'])
    fig_scatter = px.scatter(dff, x='Age', y='Performance_Score', color='Department', size='Productivity score', hover_name='Job_Title', title='Age vs Performance Score', color_discrete_sequence=px.colors.qualitative.Pastel)
    fig_box = px.box(dff, x='Department', y='Employee_Satisfaction_Score', color='Department', title='Satisfaction by Department', color_discrete_sequence=px.colors.qualitative.Vivid)
    fig_heatmap = px.density_heatmap(dff, x='Performance_Score', y='Employee_Satisfaction_Score', nbinsx=20, nbinsy=20, title='Performance vs Satisfaction Heatmap', color_continuous_scale='Viridis')

    return kpi1, kpi2, kpi3, kpi4, kpi5, kpi6, fig_donut, fig_bar, fig_line, fig_scatter, fig_box, fig_heatmap

# ---------- Status and Email Update ----------
@app.callback(
    Output('dashboard-alert', 'children'),
    Output('email-alert', 'children'),
    Input('interval-component', 'n_intervals'),
    Input('send-email-btn', 'n_clicks')
)
def update_status(n_intervals, n_clicks):
    global dashboard_status, email_status
    if ctx.triggered and 'send-email-btn' in ctx.triggered[0]['prop_id']:
        send_email()
    dashboard_status = f"📊 Dashboard updated at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    return dashboard_status, email_status

# CHATBOT SMART QUERIES (TOP N, AVERAGE, UNIQUE)

from dash import dash_table
import re
from dash import html

FIELD_SYNONYMS = {
    "age": "Age",
    "ages": "Age",
    "salary": "Annual Salary",
    "salaries": "Annual Salary",
    "annual salary": "Annual Salary",
    "department": "Department",
    "departments": "Department",
    "job title": "Job_Title",
    "job titles": "Job_Title",
    "title": "Job_Title",
    "titles": "Job_Title",
    "satisfaction": "Employee_Satisfaction_Score",
    "satisfaction score": "Employee_Satisfaction_Score",
    "performance": "Performance_Score",
    "performance": "Performance_Score",
    "performers": "Performance_Score",
    "performance score": "Performance_Score",
    "tenure": "Years_At_Company",
    "years": "Years_At_Company",
    "experience": "Years_At_Company",
    "productivity": "Productivity score",
    "productivity score": "Productivity score",
    "employee": "Employee_ID",
    "employees": "Employee_ID",
    "people": "Employee_ID",
    "staff": "Employee_ID"
}

AGGREGATION_KEYWORDS = {
    "average": "mean",
    "avg": "mean",
    "mean": "mean",
    "sum": "sum",
    "total": "count",
    "how many": "count",
    "number of": "count",
    "count": "count",
    "maximum": "max",
    "minimum": "min",
    "max": "max",
    "min": "min"
}

def resolve_field(user_input):
    user_input_lower = user_input.lower()
    for key in sorted(FIELD_SYNONYMS, key=lambda x: -len(x)):
        if key in user_input_lower:
            return FIELD_SYNONYMS[key]
    return None

def resolve_aggregation(user_input):
    user_input_lower = user_input.lower()
    for keyword, method in AGGREGATION_KEYWORDS.items():
        if keyword in user_input_lower:
            return method
    return None

def get_unique_values(df, user_input):
    for key in FIELD_SYNONYMS:
        if f"unique {key}" in user_input.lower() or f"list {key}" in user_input.lower():
            col = FIELD_SYNONYMS[key]
            values = sorted(df[col].dropna().unique())
            return html.Div([
                html.P(f"🔍 Unique values in '{col}':"),
                html.Ul([html.Li(str(val)) for val in values])
            ])
    return None

def get_aggregated_value(df, user_input):
    agg_func = resolve_aggregation(user_input)
    col = resolve_field(user_input)
    # Fallback if field is not recognized
    if not agg_func:
        return None

    if not col:
        if agg_func == "count":
            col = "Employee_ID"  # default for vague queries like "how many employees"
        else:
            return None

    if col not in df.columns:
        return None

    filtered_df = df.copy()

    # Filtering by Department or Job Title
    dept_match = re.search(r'in\s+([a-zA-Z &]+)', user_input.lower())
    if dept_match:
        keyword = dept_match.group(1).strip().lower()
        if keyword in df['Department'].str.lower().unique():
            filtered_df = df[df['Department'].str.lower() == keyword]
        elif keyword in df['Job_Title'].str.lower().unique():
            filtered_df = df[df['Job_Title'].str.lower() == keyword]

    # Year filter (e.g., after 2020, between 2019 and 2022)
    year_range = re.findall(r'(?:after|from|between|to)?\s*(\d{4})', user_input)
    if len(year_range) == 1 and 'after' in user_input.lower():
        filtered_df = filtered_df[filtered_df['Hire_Date'].dt.year > int(year_range[0])]
    elif len(year_range) == 2 and ('between' in user_input.lower() or 'from' in user_input.lower()):
        start, end = sorted(map(int, year_range))
        filtered_df = filtered_df[filtered_df['Hire_Date'].dt.year.between(start, end)]

    if filtered_df.empty:
        return html.Div("⚠️ No data available for specified filter.")

    try:
        if agg_func == "count":
            result = filtered_df[col].count()
        elif agg_func == "sum" and col == "Employee_ID":
            return html.Div("⚠️ Cannot compute sum of Employee IDs. Try asking for count instead.")
        else:
            result = getattr(filtered_df[col], agg_func)()

        if isinstance(result, (int, float)):
            result = round(result, 2)

        return html.Div(f"📊 {agg_func.title()} of '{col}' is: {result}")
    except Exception as e:
        return html.Div(f"❌ Error: {e}")

def get_top_n(df, user_input):
    match = re.search(r'top\s+(\d+)\s+([a-zA-Z _]+)', user_input.lower())
    if not match:
        return None

    n = int(match.group(1))
    metric_phrase = match.group(2).strip()

    metric_col = None
    for key, col in FIELD_SYNONYMS.items():
        if key in metric_phrase:
            metric_col = col
            break

    if not metric_col or metric_col not in df.columns:
        return html.Div(f"⚠️ Couldn't recognize the metric to rank by in: '{metric_phrase}'")

    filtered_df = df.copy()

    # Optional filtering like "top 5 performers in HR"
    dept_match = re.search(r'in\s+([a-zA-Z &]+)', user_input.lower())
    if dept_match:
        keyword = dept_match.group(1).strip().lower()
        if keyword in df['Department'].str.lower().unique():
            filtered_df = filtered_df[df['Department'].str.lower() == keyword]
        elif keyword in df['Job_Title'].str.lower().unique():
            filtered_df = filtered_df[df['Job_Title'].str.lower() == keyword]

    if filtered_df.empty:
        return html.Div("⚠️ No data available for specified filter.")

    try:
        top_df = filtered_df.nlargest(n, metric_col)[['Employee_ID', metric_col]]
        return html.Div([
            html.P(f"🏆 Top {n} by '{metric_col}':"),
            dash_table.DataTable(
                columns=[{"name": col, "id": col} for col in top_df.columns],
                data=top_df.to_dict('records'),
                style_cell={'textAlign': 'left', 'padding': '5px', 'color': 'black'},
                style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
                style_table={'overflowX': 'auto', 'border': '1px solid grey'},
            )
        ])
    except Exception as e:
        return html.Div(f"❌ Error generating top list: {e}")

def get_aggregated_value(df, user_input):
    agg_func = resolve_aggregation(user_input)
    col = resolve_field(user_input)
    if not agg_func or not col or col not in df.columns:
        return None

    filtered_df = df.copy()

    # Department or Job Title filtering
    dept_match = re.search(r'in\s+([a-zA-Z ]+)', user_input.lower())
    if dept_match:
        keyword = dept_match.group(1).strip().lower()
        if 'department' in user_input.lower() or keyword in df['Department'].str.lower().unique():
            filtered_df = filtered_df[filtered_df['Department'].str.lower() == keyword]
        elif keyword in df['Job_Title'].str.lower().unique():
            filtered_df = filtered_df[filtered_df['Job_Title'].str.lower() == keyword]

    # Year-based date filtering
    year_range = re.findall(r'(?:after|from|between|to)?\s*(\d{4})', user_input)
    if len(year_range) == 1 and 'after' in user_input.lower():
        filtered_df = filtered_df[filtered_df['Hire_Date'].dt.year > int(year_range[0])]
    elif len(year_range) == 2 and 'between' in user_input.lower():
        start, end = sorted(map(int, year_range))
        filtered_df = filtered_df[filtered_df['Hire_Date'].dt.year.between(start, end)]
    elif len(year_range) == 2 and 'from' in user_input.lower() and 'to' in user_input.lower():
        start, end = sorted(map(int, year_range))
        filtered_df = filtered_df[filtered_df['Hire_Date'].dt.year.between(start, end)]

    if filtered_df.empty:
        return f"⚠️ No data available for specified filter."

    result = getattr(filtered_df[col], agg_func)()
    return f"{agg_func.title()} of {col} is: {round(result, 2)}"

def get_groupwise_aggregation(df, user_input):
    agg_func = resolve_aggregation(user_input)

    # ✅ Improved metric column detection using whole word regex
    metric_col = None
    agg_position = user_input.lower().find(agg_func)
    best_score = float('inf')

    for key, col in FIELD_SYNONYMS.items():
        pattern = r'\b' + re.escape(key) + r'\b'
        match = re.search(pattern, user_input.lower())
        if match and col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            distance = abs(match.start() - agg_position)
            if distance < best_score:
                best_score = distance
                metric_col = col

    # Resolve group-by field
    groupby_col = None
    for key, col in FIELD_SYNONYMS.items():
        if f"by {key}" in user_input.lower() or f"in each {key}" in user_input.lower():
            groupby_col = col
            break
        # handle phrasing like "salary of each department"
        elif f"each {key}" in user_input.lower():
            groupby_col = col
            break

    if not agg_func or not metric_col or not groupby_col:
        return None

    if metric_col not in df.columns or groupby_col not in df.columns:
        return None

    try:
        df[metric_col] = pd.to_numeric(df[metric_col], errors='coerce')

        grouped = df.groupby(groupby_col, as_index=False)[metric_col].agg(agg_func)
        grouped[metric_col] = grouped[metric_col].round(2)

        return html.Div([
            html.P(f"📊 {agg_func.title()} of {metric_col} by {groupby_col}:"),
            dash_table.DataTable(
                columns=[{"name": col, "id": col} for col in grouped.columns],
                data=grouped.to_dict('records'),
                style_cell={'textAlign': 'left', 'padding': '5px', 'color': 'black'},
                style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
                style_table={'overflowX': 'auto', 'border': '1px solid grey'},
            )
        ])
    except Exception as e:
        return html.Div(f"❌ Error: {e}")

@app.callback(
    Output('chat-response', 'children'),
    Output('chat-input', 'value'),
    Output('faq-dropdown', 'value'),
    Input('chat-submit-btn', 'n_clicks'),
    Input('faq-dropdown', 'value'),
    Input('chat-clear-btn', 'n_clicks'),
    State('chat-input', 'value'),
    prevent_initial_call=True
)
def handle_chat(submit_clicks, selected_question, clear_clicks, user_input):
    if ctx.triggered_id == 'chat-clear-btn':
        return html.Div("Welcome! Ask me anything about the dashboard."), "", None

    if ctx.triggered_id == 'faq-dropdown' and selected_question:
        answer = predefined_qa.get(selected_question, "I'm not sure about that.")
        return html.Div(answer), "", selected_question

    if ctx.triggered_id == 'chat-submit-btn' and user_input:
        dff = load_data()

        # 1. Check predefined Q&A
        for q, a in predefined_qa.items():
            if user_input.lower() in q.lower():
                return html.Div(a), "", None

        # 2. Smart queries
        smart_response = (
            get_unique_values(dff, user_input) or
            get_top_n(dff, user_input) or
            get_groupwise_aggregation(dff, user_input) or
            get_aggregated_value(dff, user_input)
        )
        
        if smart_response:
            return smart_response, "", None

        # 3. Fallback
        return html.Div("❓ Sorry, I don't have an answer for that yet. Try asking about averages, top performers, or listing fields."), "", None

    return dash.no_update

# ---------- Run ----------
app.run(debug=True,port=8055)
